In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
# We create our model
model = gp.Model('Promotions')

Academic license - for non-commercial use only - expires 2022-09-25
Using license file /Users/annary1996/gurobi.lic


In [3]:
disc = [1, 0.9, 0.8, 0.6]
demand_rate = [1.0, 1.36, 1.79, 2.71]
price = 60
initial_demand = 73
d = list((demand_rate*np.array(initial_demand)).round())
units = 2000

In [4]:
d

[73.0, 99.0, 131.0, 198.0]

In [5]:
p = model.addVars(15, vtype = GRB.CONTINUOUS, name = ["price_"+str(i) for i in range(1,16)])
d_ind = model.addVars(15, 4, vtype = GRB.BINARY, name = "decide_price_")
sale = model.addVars(15, vtype = GRB.CONTINUOUS, name = ["demand_"+str(i) for i in range(1,16)])

In [6]:
rev = sum(p[i]*sale[i] for i in range(15))

model.setObjective(rev, GRB.MAXIMIZE)

In [7]:
#model.addConstr(d[0]==initial_demand)
#model.addConstr(y[0]==initial_demand)
model.addConstr(p[0]==60)

for i in range(15):
    model.addConstr(sum(d_ind[i,j] for j in range(4))==1, name = "Only_one_discount_"+str(i))
    model.addConstr(sale[i]<=sum(d_ind[i,j]*d[j] for j in range(4)), name = "Sales_constriant"+str(i))
    
for i in range(14):
    model.addConstr(sum(d_ind[i,j]*disc[j]*price for j in range(4))>=sum(d_ind[i+1,j]*disc[j]*price for j in range(4)), name = "maintain_or_reduce_discount_"+str(i))
    model.addConstr(p[i+1]==sum(d_ind[i+1,j]*disc[j]*p[0] for j in range(4)), name = "price_"+str(i))    

In [8]:
model.addConstr(sum(d_ind[i,j]*d[j] for j in range(4) for i in range(15))<=units, name = "Total_stock_bounding")
model.addConstr(d_ind[0,0]==1)
model.addConstr(p[0]==60)

<gurobi.Constr *Awaiting Model Update*>

In [10]:
model.params.NonConvex = 2
model.optimize()

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 48 rows, 90 columns and 310 nonzeros
Model fingerprint: 0x37153c90
Model has 15 quadratic objective terms
Model has 14 quadratic constraints
Variable types: 30 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [6e-01, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve added 8 rows and 0 columns
Presolve removed 0 rows and 13 columns
Presolve time: 0.00s
Presolved: 85 rows, 92 columns, 331 nonzeros
Presolved model has 14 bilinear constraint(s)
Variable types: 29 continuous, 63 integer (49 binary)

Root relaxation: objective 9.901200e+04, 99 iterations, 0.00 sec

In [11]:
if model.status == GRB.OPTIMAL:
    model.printAttr('X')


    Variable            X 
-------------------------
     price_1           60 
     price_2           48 
     price_3           48 
     price_4           48 
     price_5           48 
     price_6           48 
     price_7           48 
     price_8           48 
     price_9           48 
    price_10           48 
    price_11           48 
    price_12           48 
    price_13           48 
    price_14           48 
    price_15           36 
decide_price_[0,0]            1 
decide_price_[1,2]            1 
decide_price_[2,2]            1 
decide_price_[3,2]            1 
decide_price_[4,2]            1 
decide_price_[5,2]            1 
decide_price_[6,2]            1 
decide_price_[7,2]            1 
decide_price_[8,2]            1 
decide_price_[9,2]            1 
decide_price_[10,2]            1 
decide_price_[11,2]            1 
decide_price_[12,2]            1 
decide_price_[13,2]            1 
decide_price_[14,3]            1 
    demand_1           73 
    demand_2  

## Wrap this into a function

In [7]:
def markdown_strategy():
    input_price = input("Input initial price: ")
    price = int(input_price)
    input_initial_sales = input("Input initial sales: ")
    initial_demand = int(input_initial_sales)
    disc = [1, 0.9, 0.8, 0.6]
    demand_rate = [1.0, 1.36, 1.79, 2.71]
    d = list((demand_rate*np.array(initial_demand)).round())
    units = 2000
    ## Establish Gurobi model
    model_gb = gp.Model('Promotions')
    p = model_gb.addVars(15, vtype = GRB.CONTINUOUS, name = ["price_"+str(i) for i in range(1,16)])
    d_ind = model_gb.addVars(15, 4, vtype = GRB.BINARY, name = "decide_price_")
    sale = model_gb.addVars(15, vtype = GRB.CONTINUOUS, name = ["demand_"+str(i) for i in range(1,16)])
    
    rev = sum(p[i]*sale[i] for i in range(15))
    model_gb.setObjective(rev, GRB.MAXIMIZE)
    
    ## Add constraints
    model_gb.addConstr(p[0]==price)
    model_gb.addConstr(d_ind[0,0]==1)

    for i in range(15):
        model_gb.addConstr(sum(d_ind[i,j] for j in range(4))==1, name = "Only_one_discount_"+str(i))
        model_gb.addConstr(sale[i]<=sum(d_ind[i,j]*d[j] for j in range(4)), name = "Sales_constriant"+str(i))
    
    for i in range(14):
        model_gb.addConstr(sum(d_ind[i,j]*disc[j]*price for j in range(4))>=sum(d_ind[i+1,j]*disc[j]*price for j in range(4)), name = "maintain_or_reduce_discount_"+str(i))
        model_gb.addConstr(p[i+1]==sum(d_ind[i+1,j]*disc[j]*p[0] for j in range(4)), name = "price_"+str(i))
    
    model_gb.addConstr(sum(d_ind[i,j]*d[j] for j in range(4) for i in range(15))<=units, name = "Total_stock_bounding")
    
    ## Solve
    model_gb.params.NonConvex = 2
    model_gb.optimize()
    
    if model_gb.status == GRB.OPTIMAL:
        print("**Strategy:**")
        model_gb.printAttr('X')
    

In [15]:
markdown_strategy()

Input initial price: 70
Input initial sales: 65
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 47 rows, 90 columns and 309 nonzeros
Model fingerprint: 0x655dbdd1
Model has 15 quadratic objective terms
Model has 14 quadratic constraints
Variable types: 30 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [6e-01, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve added 9 rows and 0 columns
Presolve removed 0 rows and 11 columns
Presolve time: 0.00s
Presolved: 85 rows, 94 columns, 343 nonzeros
Presolved model has 14 bilinear constraint(s)
Variable types: 29 continuous, 65 integer (51 binary)

Root relaxation:

In [10]:
markdown_strategy()

Input initial price: 60
Input initial sales: 44
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 47 rows, 90 columns and 309 nonzeros
Model fingerprint: 0x5e09e6fb
Model has 15 quadratic objective terms
Model has 14 quadratic constraints
Variable types: 30 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  QMatrix range    [6e-01, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve added 8 rows and 0 columns
Presolve removed 0 rows and 6 columns
Presolve time: 0.00s
Presolved: 84 rows, 99 columns, 331 nonzeros
Presolved model has 14 bilinear constraint(s)
Variable types: 29 continuous, 70 integer (56 binary)

Root relaxation: 

In [11]:
markdown_strategy()

Input initial price: 60
Input initial sales: 93
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 47 rows, 90 columns and 309 nonzeros
Model fingerprint: 0x2c6616cf
Model has 15 quadratic objective terms
Model has 14 quadratic constraints
Variable types: 30 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  QMatrix range    [6e-01, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 1 rows and 32 columns
Presolve time: 0.00s
Presolved: 74 rows, 73 columns, 251 nonzeros
Presolved model has 9 bilinear constraint(s)
Variable types: 29 continuous, 44 integer (35 binary)

Root relaxation: objective 1.111439e+05, 85 iteration

In [12]:
markdown_strategy()

Input initial price: 60
Input initial sales: 73
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 47 rows, 90 columns and 309 nonzeros
Model fingerprint: 0xe645897a
Model has 15 quadratic objective terms
Model has 14 quadratic constraints
Variable types: 30 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [6e-01, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve added 9 rows and 0 columns
Presolve removed 0 rows and 13 columns
Presolve time: 0.00s
Presolved: 85 rows, 92 columns, 331 nonzeros
Presolved model has 14 bilinear constraint(s)
Variable types: 29 continuous, 63 integer (49 binary)

Root relaxation: